## Connecting with Google drive

In [1]:
from google.colab import drive

drive.mount('/content/drive')
#root_path = 'drive/My Drive/Lab/'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Select current working directory or folder

In [2]:
import os
print("Current working directory: "+os.getcwd())
default_path = '/content/drive/My Drive/Lab/API'
os.chdir(default_path)
print("Current working directory: "+os.getcwd())

Current working directory: /content
Current working directory: /content/drive/My Drive/Lab/API


## Import Support files and packages

In [3]:
from utils import *
from image_transform import ImageTransform
from loader import Loader
from graph_plot import *
from test import TestModel
from train import TrainModel
from display_images import *
from train_network import *
from cutout import Cutout
from albumentation_transform import AlbumentationTransformations


# from resnet import *
# from albumentations import *
# from albumentations.pytorch.transforms import ToTensor

In [4]:
IMAGE_PATH = "/content/drive/My Drive/Lab/s11/"
PATH = "/content/drive/My Drive/Lab/s11"
MODEL_PATH = "/content/drive/My Drive/Lab/s11/"

train_losses = []
train_accuracy = []
test_losses = []
test_accuracy = []

## Setup device

In [5]:
# Device
SEED = 1
cuda = torch.cuda.is_available()
print("Cuda is available ?", cuda)
torch.manual_seed(SEED)
if cuda:
    torch.cuda.manual_seed(SEED)
device = torch.device("cuda" if cuda else "cpu")

Cuda is available ? True


## Load DevidNet model

In [6]:
from devidnet_architecture import resnet20
model = resnet20().to(device)
summary(model, input_size=(3, 32, 32))

NameError: ignored

## Model Summary (Layers and Parameter)

In [ ]:
model

## Data augmentation, Dataset, DataLoader

In [ ]:
img_mean = (0.4914, 0.4822, 0.4465)
img_std = (0.2023, 0.1994, 0.2010)

# Transforms
means = np.array(img_mean)
stdevs = np.array(img_std)
patch_size = 32

# Define Train transforms and Test transforms

transform_train = [  
                   transforms.RandomHorizontalFlip(),
                   transforms.RandomCrop(32, padding=4),          
                   transforms.ToTensor(),
                   transforms.Normalize(mean=means, std=stdevs),
                   Cutout(1,8),    
    ]

transform_test = [transforms.ToTensor(),
                  transforms.Normalize(mean=means, std=stdevs),
         
    ]

# Create Train transforms and Test transforms
t_transform_train =  ImageTransform.transform(transform_train)
t_transform_test = ImageTransform.transform(transform_test)

# Dataset and DataLoader arguments
dataset_name = torchvision.datasets.CIFAR10
trainSet_dict = dict(root='./data', train=True, download=True, transform=t_transform_train)
trainLoad_dict = dict(batch_size=512, shuffle=True, num_workers=4)
testSet_dict = dict(root='./data', train=False, download=True, transform=t_transform_test)
testLoad_dict = dict(batch_size=32, shuffle=False, num_workers=4)


# Create Train Loader and Test Loader
trainloader = Loader.getDataLoader(dataset_name, trainSet_dict, trainLoad_dict)
testloader = Loader.getDataLoader(dataset_name, testSet_dict, testLoad_dict)
classes = ('plane', 'car', 'bird', 'cat',
            'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

## Loss Function, Optimizer and LR Scheduler

In [ ]:
# model = ResNet18().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-07, weight_decay=1e-2)

## Best Learning Rate Finder

In [ ]:
from find_lr_max import *
lrMax = get_LR(model, trainloader, optimizer, criterion, device)

## Hyperparameters

In [ ]:
# Hyper parameters
LAMBDA1 = 1e-5
LR = lrMax
MOMENTUM = (0.95, 0.85)
WEIGHT_DECAY = 1e-2  #
LR_EPOCHS = 5
Max_EPOCHS = 24
DIV_FACTOR = 10.0
START_PERCENT = 0.0353 #LR_EPOCHS/Max_EPOCHS  #0.135 

## Train network for few epochs to find best learning rate and save the model

In [ ]:
lr_list, moms_list, train_losses, train_accuracy, test_losses, test_accuracy, optimizer = trainNetwork(model,device, trainloader, testloader, optimizer, criterion, epochs=5, lr_max=LR, moms=MOMENTUM, div_factor=DIV_FACTOR, pct_start=START_PERCENT,
                 MODEL_PATH=MODEL_PATH, L1_regularization=None, WEIGHT_DECAY= WEIGHT_DECAY)

## Plot learning rate cycle scheduler

In [ ]:
from cyclic_learning_rate import *
lr_min, lr_max, step_size, max_iteration, path, name = 0.08, 0.8, 10, 100, PATH+"/visualization", "clr_graph"
generate_cyclic_learning_rate(lr_min, lr_max, step_size, max_iteration, path, name)

In [ ]:
PATH = "/content/drive/My Drive/Lab/s11"

In [ ]:
from graph_plot import *
plot_lr(lr_list,name="lr_plot", PATH=PATH+str("/visualization/"))

## Load model

In [ ]:
filename = MODEL_PATH + "S11_model.pth"

In [ ]:
def load_checkpoint(model, optimizer,  filename='checkpoint.pth.tar'):
    # Note: Input model & optimizer should be pre-defined.  This routine only updates their states.
    start_epoch = 0
    if os.path.isfile(filename):
        print("=> loading checkpoint '{}'".format(filename))
        checkpoint = torch.load(filename)
        start_epoch = checkpoint['epoch']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        # losslogger = checkpoint['losslogger']
        print("=> loaded checkpoint '{}' (epoch {})"
                  .format(filename, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(filename))

    return model, optimizer, start_epoch

In [ ]:
#model = ResNet18().to(device)
model, optimizer, start_epoch = load_checkpoint(model, optimizer,  filename)

In [ ]:
optimizer.param_groups[-1]["lr"] = optimizer.param_groups[-1]["lr"]/10
# optimizer.param_groups[-1]["weight_decay"] = 1e-3

# Training  network

In [ ]:
# Train more
for epoch in range(start_epoch, start_epoch+Max_EPOCHS):
  train_loss, train_acc = TrainModel.train(model, device, trainloader, criterion, optimizer, epoch)
  # scheduler.step()
  train_losses.append(train_loss)
  train_accuracy.append(train_acc)
  test_loss, test_acc = TestModel.test(model, device, testloader, criterion)
  test_losses.append(test_loss)
  test_accuracy.append(test_acc)

  state = {'epoch': epoch + 1, 'state_dict': model.state_dict(),
             'optimizer': optimizer.state_dict()}
  torch.save(state, filename)

## Show and Save losses and accuracy Plot

In [ ]:
getPlottedGraph(Max_EPOCHS, train_losses, train_accuracy, test_losses, test_accuracy,name="S11_plot", PATH=PATH+str("/visualization/"))

## Load Model

In [ ]:
def load_checkpoint(model, optimizer,  filename='checkpoint.pth.tar'):
    # Note: Input model & optimizer should be pre-defined.  This routine only updates their states.
    start_epoch = 0
    if os.path.isfile(filename):
        print("=> loading checkpoint '{}'".format(filename))
        checkpoint = torch.load(filename)
        start_epoch = checkpoint['epoch']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        # losslogger = checkpoint['losslogger']
        print("=> loaded checkpoint '{}' (epoch {})"
                  .format(filename, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(filename))

    return model, optimizer, start_epoch

In [ ]:
model = resnet20().to(device)
model, optimizer, start_epoch = load_checkpoint(model, optimizer,  filename)

## Show and Save correct classified images

In [ ]:
show_save_correctly_classified_images(model, testloader, device, PATH, name="correct_classified_imgs", max_correctly_classified_images_imgs=25,labels_list=classes)

## Show and Save misclassified images

In [ ]:
show_save_misclassified_images(model, testloader, device, PATH, name="misclassified_imgs", max_misclassified_imgs=25,labels_list=classes)

## Grad Cam

### Top 5 Correct classified image GradCam 

In [ ]:
misclassified_imgs, correctly_classified_images = classify_images(model, testloader, device, 25)

In [ ]:
from grad_cam import *

In [ ]:
layers_list = ["layer1","layer2","layer3"]
display_gradcam = VisualizeCam(model, classes, layers_list)

### Top 5 misclassifeid image GradCam 

In [ ]:
correct_pred_imgs = []
for i in range(len(correctly_classified_images)):
  correct_pred_imgs.append(torch.as_tensor(correctly_classified_images[i]["img"]))
display_gradcam(torch.stack(correct_pred_imgs), layers_list, PATH+str("/visualization"), metric="correct")

In [ ]:
misclassify_pred_imgs = []
for i in range(len(misclassified_imgs)):
  misclassify_pred_imgs.append(torch.as_tensor(misclassified_imgs[i]["img"]))
display_gradcam(torch.stack(misclassify_pred_imgs), layers_list, PATH+str("/visualization"), metric="misclassify")

### 25 misclassified images GradCam

In [ ]:
display_gradcam.plot_img_heatmap(misclassified_imgs, layers_list, PATH+"/visualization", target_inds=None, metric="", name="gradcam_misclassified_images")

In [ ]:
# Train more
for epoch in range(start_epoch, start_epoch+2):
  train_loss, train_acc = TrainModel.train(model, device, trainloader, criterion, optimizer, epoch)
  # scheduler.step()
  train_losses.append(train_loss)
  train_accuracy.append(train_acc)
  test_loss, test_acc = TestModel.test(model, device, testloader, criterion)
  test_losses.append(test_loss)
  test_accuracy.append(test_acc)

  state = {'epoch': epoch + 1, 'state_dict': model.state_dict(),
             'optimizer': optimizer.state_dict()}
  torch.save(state, filename)